In [73]:
# naver maps api id, key
import yaml

with open("config.yaml", "r") as file:
    config = yaml.safe_load(file)

naver_api_id = config['naver api']['id']
naver_api_key = config['naver api']['key']
kakao_api_key = config['kakao api']['key']

In [74]:
import os
import pandas as pd

# 'data/raw_data' 경로 설정
root_dir = 'data/'
visitor_city = pd.read_csv(os.path.join(root_dir, "city_of_festival_visitors.csv"))  # 무주축제 방문객 top 18 지역들 (시군구명,od_cnts,시도명,행정동코드,위도,경도)
address = pd.read_csv(os.path.join(root_dir, "address_with_lon_lat_final.csv"))  # 행정동코드 + 위도경도 (행정동코드,시도명,시군구명,읍면동명,동리명,위도,경도)
mooju = set(list(address[address['시군구명'] == '무주군']['행정동코드']))  # 무주군 행정동코드
other_city = list(address.merge(visitor_city, on=['시도명', '시군구명'])['행정동코드_x'])  # 다른 지역들 행정동코드 모음

In [75]:
address['시도 시군구'] = address['시도명'].fillna('') + ' ' + address['시군구명'].fillna('')

# 양쪽 값이 모두 null인 경우 빈 문자열 처리
address['시도 시군구'] = address['시도 시군구'].str.strip()
address.head()

,행정동코드,시도명,시군구명,읍면동명,동리명,위도,경도,시도 시군구
0,1100000000,서울특별시,NaN,NaN,서울특별시,37.566679,126.978291,서울특별시
1,1111000000,서울특별시,종로구,NaN,종로구,37.580695,126.982799,서울특별시 종로구
2,1111051500,서울특별시,종로구,청운효자동,세종로,37.579997,126.976930,서울특별시 종로구
3,1111051500,서울특별시,종로구,청운효자동,옥인동,37.583480,126.963850,서울특별시 종로구
4,1111051500,서울특별시,종로구,청운효자동,누하동,37.578998,126.967561,서울특별시 종로구


In [76]:
def get_coordinates(lst):
    coordinates = []
    for name in lst:
        try:
            # '시도 시군구'로 '위도', '경도' 가져옴
            target = address[address['시도 시군구'] == name][['위도','경도']].iloc[0]
            x, y = target.iloc[0], target.iloc[1]
            coordinates.append((x,y))
        except IndexError:
            # 해당 '시도 시군구'에 대한 데이터가 없는 경우
            print(f"{name} 지역의 위도, 경도 정보가 없습니다.")
    return coordinates


In [77]:
import requests

def naver_request(start, goal):
    # 요청
    url = 'https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving'
    params = {
        'goal': f'{goal[1]},{goal[0]}',
        'start': f'{start[1]},{start[0]}',
    }
    headers = {
        'x-ncp-apigw-api-key-id': naver_api_id,
        'x-ncp-apigw-api-key': naver_api_key
    }
    response = requests.get(url, headers=headers, params=params)

    # 응답
    data = response.json()
    if data['code'] == 0:
        if data['route']['traoptimal']:
            summary = data['route']['traoptimal'][0]['summary']
            distance = summary['distance']
            duration = summary['duration']
            return distance, duration
        else:
            print("응답 에러: 'traoptimal' 데이터가 없습니다.")
            return -1, -1
    else:
        print("요청 실패: ", data['message'])
        return -1, -1


def kakao_request(start, goal):
    # 요청
    url = 'https://apis-navi.kakaomobility.com/v1/directions'
    params = {
        'origin': f'{start[1]},{start[0]}',
        'destination': f'{goal[1]},{goal[0]}',
        'alternatives': True,
    }
    headers = {
        'Authorization': f'KakaoAK {kakao_api_key}'
    }
    response = requests.get(url, headers=headers, params=params)

    # 응답
    if response.status_code == 200:
        data = response.json()
        if data['routes']:
            summary = data['routes'][0]['summary']
            distance = summary.get('distance')
            duration = summary.get('duration')
            return distance, duration
        else:
            print("응답 에러: 'routes' 데이터가 없습니다.")
            return -1, -1
    else:
        print("요청 실패")
        return -1, -1


def get_path_info(coordinates, request_func):
    n = len(coordinates)
    distance_table = [[0] * n for _ in range(n)]
    duration_table = [[0] * n for _ in range(n)]

    for i in range(n):
        for j in range(n):
            if j > i:
                start = coordinates[i]
                goal = coordinates[j]
                distance, duration = request_func(start, goal)
                distance_table[i][j], distance_table[j][i] = distance, distance
                duration_table[i][j], duration_table[j][i] = duration, duration

    return distance_table, duration_table

---

In [78]:
# 경유지 설정
daejun = ['세종특별자치시', '대전광역시 유성구', '대전광역시 서구', '대전광역시 대덕구', '대전광역시 중구', '충청남도 금산군', '충청북도 영동군', '전라북도 무주군']
jeonbuk = ['전라북도 군산시', '전라북도 익산시', '전라북도 전주시 완산구', '전라북도 전주시 덕진구', '전라북도 진안군', '전라북도 장수군', '전라북도 무주군']

nodes = daejun  # ← 여기 변경 

In [79]:
coordinates = get_coordinates(nodes)  # 경유지 위경도 좌표 받아옴
coordinates

[(36.4799999, 127.289),
 (36.3620732, 127.3564099),
 (36.3551786, 127.3838487),
 (36.3465766, 127.4158483),
 (36.3254513, 127.4211737),
 (36.1087249, 127.48815),
 (36.1747588, 127.7834432),
 (36.007, 127.6609)]

In [81]:
# 위경도 좌표를 기반으로 노드 간 거리와 시간 정보 구하기
# - api 호출하므로 마구 누르는 거 주의 !

# 네이버
n_distance_table, n_duration_table = get_path_info(coordinates, naver_request)

In [89]:
# 카카오
k_distance_table, k_duration_table = get_path_info(coordinates, kakao_request)

In [90]:
k_distance_table

[[0, 17501, 20307, 25972, 25261, 59883, 72701, 83433],
 [17501, 0, 3940, 7631, 9274, 38037, 54341, 71682],
 [20307, 3940, 0, 4149, 6328, 35042, 51309, 66681],
 [25972, 7631, 4149, 0, 3420, 31509, 47687, 63059],
 [25261, 9274, 6328, 3420, 0, 29509, 49193, 54068],
 [59883, 38037, 35042, 31509, 29509, 0, 38628, 31299],
 [72701, 54341, 51309, 47687, 49193, 38628, 0, 27200],
 [83433, 71682, 66681, 63059, 54068, 31299, 27200, 0]]

In [91]:
k_duration_table

[[0, 1433, 1619, 1700, 2232, 3240, 4334, 4106],
 [1433, 0, 577, 848, 1002, 3037, 3935, 3456],
 [1619, 577, 0, 690, 894, 2940, 3838, 3601],
 [1700, 848, 690, 0, 481, 2451, 3435, 3193],
 [2232, 1002, 894, 481, 0, 2171, 3562, 3133],
 [3240, 3037, 2940, 2451, 2171, 0, 3011, 1963],
 [4334, 3935, 3838, 3435, 3562, 3011, 0, 1625],
 [4106, 3456, 3601, 3193, 3133, 1963, 1625, 0]]

In [94]:
k_duration_min_table = [[round(value / 60, 2) for value in row] for row in k_duration_table]
k_duration_min_table

[[0.0, 23.88, 26.98, 28.33, 37.2, 54.0, 72.23, 68.43],
 [23.88, 0.0, 9.62, 14.13, 16.7, 50.62, 65.58, 57.6],
 [26.98, 9.62, 0.0, 11.5, 14.9, 49.0, 63.97, 60.02],
 [28.33, 14.13, 11.5, 0.0, 8.02, 40.85, 57.25, 53.22],
 [37.2, 16.7, 14.9, 8.02, 0.0, 36.18, 59.37, 52.22],
 [54.0, 50.62, 49.0, 40.85, 36.18, 0.0, 50.18, 32.72],
 [72.23, 65.58, 63.97, 57.25, 59.37, 50.18, 0.0, 27.08],
 [68.43, 57.6, 60.02, 53.22, 52.22, 32.72, 27.08, 0.0]]

---

In [95]:
import numpy as np
import networkx as nx

# 모든 경로 찾기
def find_all_routes(G, start, target, visited=None, path=None):
    if visited is None:
        visited = set()
    if path is None:
        path = []

    visited.add(start)
    path.append(start)

    if start == target:
        yield path.copy()
    else:
        for neighbor in G.neighbors(start):
            if neighbor not in visited:
                yield from find_all_routes(G, neighbor, target, visited, path)

    path.pop()
    visited.remove(start)

# 최적 경로 탐색 함수
def find_optimal_routes_within_range(G, stations, min_factor, max_factor, step):
    optimal_routes = []
    max_stops = 0
    explored_routes = set()  # 탐색된 경로를 저장할 집합

    # 여러 배수 값을 탐색
    for factor in np.arange(min_factor, max_factor + step, step):

        for station in stations.keys():
            if station != "전라북도 무주군":  # 목적지가 아니면
                routes = list(find_all_routes(G, station, "전라북도 무주군"))

                for route in routes:
                    valid_route = True
                    total_distance = 0
                    total_time = 0

                    # 각 경로의 모든 구간에 대해 확인
                    for i in range(len(route) - 1):
                        current_station = route[i]

                        # 현재 노드에서 무주군으로 바로 이동할 때의 거리와 시간
                        if "전라북도 무주군" in G[current_station]:  # 직접 경로가 있는 경우에만
                            direct_distance = G[current_station]["전라북도 무주군"]['distance']
                            direct_time = G[current_station]["전라북도 무주군"]['time']

                            # 현재 구간의 거리와 시간을 누적
                            total_distance += G[route[i]][route[i + 1]]['distance']
                            total_time += G[route[i]][route[i + 1]]['time']

                            # 각 중간 노드에서도 주어진 배수(factor) 조건 확인
                            if total_distance > direct_distance * factor or total_time > direct_time * factor:
                                valid_route = False
                                break

                    if valid_route:
                        # 경로를 집합에 저장할 수 있는 형태로 변환 (정렬하여 순서 상관없이 비교 가능)
                        route_tuple = tuple(sorted(route))

                        # 만약 해당 경로가 이전 배수에서 이미 탐색되었다면 제외
                        if route_tuple not in explored_routes:
                            explored_routes.add(route_tuple)  # 새로운 경로로 저장
                            # 가장 많은 정거장 수 확인
                            if len(route) > max_stops:
                                max_stops = len(route)
                                optimal_routes = [(route, total_distance, total_time, factor)]
                            elif len(route) == max_stops:
                                optimal_routes.append((route, total_distance, total_time, factor))

    return optimal_routes

In [97]:
# 정거장 후보지와 가중치
stations = {
    "세종특별자치시": 1,
    "대전광역시 유성구": 2,  # 가중치 2로 증가
    "대전광역시 서구": 2,    # 가중치 2로 증가
    "대전광역시 대덕구": 1,
    "대전광역시 중구": 1,
    "충청남도 금산군": 1,
    "충청북도 영동군": 2,    # 가중치 2로 증가
    "전라북도 무주군": 0,    # 목적지
}

# 그래프 생성
G = nx.DiGraph()

# 그래프에 엣지 추가
for i, from_station in enumerate(stations.keys()):
    for j, to_station in enumerate(stations.keys()):
        if i != j:
            G.add_edge(from_station, to_station, 
                       distance=k_distance_table[i][j], 
                       time=k_duration_min_table[i][j])
            
# 탐색 범위 설정 (예: 1.1부터 1.5까지 0.1 간격으로 탐색)
min_factor = 1.1
max_factor = 1.5
step = 0.1

# 최적 경로 탐색 실행
optimal_routes = find_optimal_routes_within_range(G, stations, min_factor, max_factor, step)

# 결과 출력
print("충남지역 최적의 셔틀 노선")
if optimal_routes:
    for route, total_distance, total_time, factor in optimal_routes:
        print(f"최적의 셔틀 노선: {route} | 총 거리: {total_distance} | 총 시간: {total_time} | 배수: {factor:.2f}")
else:
    print("유효한 경로가 없습니다.")

충남지역 최적의 셔틀 노선
최적의 셔틀 노선: ['대전광역시 유성구', '대전광역시 서구', '대전광역시 대덕구', '전라북도 무주군'] | 총 거리: 71148 | 총 시간: 74.34 | 배수: 1.40
최적의 셔틀 노선: ['대전광역시 서구', '대전광역시 대덕구', '대전광역시 중구', '전라북도 무주군'] | 총 거리: 61637 | 총 시간: 71.74 | 배수: 1.40
최적의 셔틀 노선: ['대전광역시 중구', '대전광역시 대덕구', '대전광역시 유성구', '전라북도 무주군'] | 총 거리: 82733 | 총 시간: 79.75 | 배수: 1.40
최적의 셔틀 노선: ['대전광역시 유성구', '대전광역시 서구', '대전광역시 중구', '전라북도 무주군'] | 총 거리: 64336 | 총 시간: 76.74 | 배수: 1.50
최적의 셔틀 노선: ['대전광역시 서구', '대전광역시 유성구', '세종특별자치시', '전라북도 무주군'] | 총 거리: 104874 | 총 시간: 101.93 | 배수: 1.50


In [99]:
nodes = jeonbuk  # ← 여기 변경 

coordinates = get_coordinates(nodes)  # 경유지 위경도 좌표 받아옴

jb_k_distance_table, jb_k_duration_table = get_path_info(coordinates, kakao_request)

jb_k_duration_min_table = [[round(value / 60, 2) for value in row] for row in jb_k_duration_table]

# 정거장 후보지와 가중치
stations = {
    '전라북도 군산시': 1, 
    '전라북도 익산시': 1, 
    '전라북도 전주시 완산구': 2, 
    '전라북도 전주시 덕진구': 2, 
    '전라북도 진안군': 1, 
    '전라북도 장수군': 2,
    '전라북도 무주군': 0,    # 목적지
}

# 그래프 생성
G = nx.DiGraph()

# 그래프에 엣지 추가
for i, from_station in enumerate(stations.keys()):
    for j, to_station in enumerate(stations.keys()):
        if i != j:
            G.add_edge(from_station, to_station, 
                       distance=jb_k_distance_table[i][j], 
                       time=jb_k_duration_min_table[i][j])
            
# 탐색 범위 설정 (예: 1.1부터 1.5까지 0.1 간격으로 탐색)
min_factor = 1.1
max_factor = 1.5
step = 0.1

# 최적 경로 탐색 실행
optimal_routes = find_optimal_routes_within_range(G, stations, min_factor, max_factor, step)

# 결과 출력
print("전북지역 최적의 셔틀 노선")
if optimal_routes:
    for route, total_distance, total_time, factor in optimal_routes:
        print(f"최적의 셔틀 노선: {route} | 총 거리: {total_distance} | 총 시간: {total_time} | 배수: {factor:.2f}")
else:
    print("유효한 경로가 없습니다.")

전북지역 최적의 셔틀 노선
최적의 셔틀 노선: ['전라북도 전주시 완산구', '전라북도 전주시 덕진구', '전라북도 익산시', '전라북도 무주군'] | 총 거리: 124344 | 총 시간: 137.02 | 배수: 1.50
